In [1]:
# Import pandas
import pandas as pd

In [2]:
# Get initial data from NVSim
with open("nvsim/nvsim.csv") as nvsim:
  data = pd.read_csv(nvsim)
data

,mem,opt,banks,tREAD,tWRITE,eREAD,eWRITE,pLEAK,area
0,PCRAM,Area,1,66.918000,105.704,5.219570e-10,6.521000e-09,0.027884,0.309007
1,STTRAM,ReadEDP,32,2.884000,10.922,5.070100e-11,1.021850e-10,0.568334,2.204000
2,STTRAM,ReadDynamicEnergy,8,79.992000,88.485,2.876800e-11,7.480400e-11,0.093780,1.934000
3,RRAM,ReadDynamicEnergy,8,4.672000,23.730,2.660100e-11,1.142810e-10,0.281180,0.352296
4,STTRAM,Area,1,314.360000,322.387,5.736900e-11,8.109200e-11,0.027903,1.786000
5,RRAM,ReadEDP,64,2.159000,20.670,3.099200e-11,6.588200e-11,0.377108,0.465897
6,STTRAM,ReadLatency,512,2.659000,10.686,1.145980e-10,1.603930e-10,1.583000,2.503000
7,PCRAM,ReadDynamicEnergy,64,4.542000,44.335,1.695300e-11,6.494000e-09,0.255420,0.467861
8,RRAM,Area,4,11.586000,30.432,1.163350e-10,2.025640e-10,0.162125,0.202552
9,PCRAM,ReadEDP,128,0.486710,40.304,2.103000e-11,6.502000e-09,0.473781,0.549767


In [4]:
# Get additional data from gem5
for i, row in data.iterrows():
  gem5data = pd.read_csv(f"gem5/{row['mem']}/{row['opt']}/stats.txt", delimiter=r"\s+", skiprows=2, skipfooter=1, usecols=[0,1], names=["stat", "value"], engine="python")
  data.at[i, "runtime"] = gem5data[gem5data.stat == "simSeconds"].value.values[0]
  data.at[i, "nREAD"] = gem5data[gem5data.stat == "system.mem_ctrls.readReqs"].value.values[0]
  data.at[i, "nWRITE"] = gem5data[gem5data.stat == "system.mem_ctrls.writeReqs"].value.values[0]
  data.at[i, "eREADs"] = row["eREAD"] * data.at[i, "nREAD"]
  data.at[i, "eWRITEs"] = row["eWRITE"] * data.at[i, "nWRITE"]
  data.at[i, "eLEAK"] = row["pLEAK"] * data.at[i, "runtime"]
  data.at[i, "eTOTAL"] = data.at[i, "eREADs"] + data.at[i, "eWRITEs"] + data.at[i, "eLEAK"]
data.to_csv("analysis.csv")
data

,mem,opt,banks,tREAD,tWRITE,eREAD,eWRITE,pLEAK,area,runtime,nREAD,nWRITE,eREADs,eWRITEs,eLEAK,eTOTAL
0,PCRAM,Area,1,66.918000,105.704,5.219570e-10,6.521000e-09,0.027884,0.309007,4.971777,13422248.0,1251051.0,0.007006,0.008158,0.138633,0.153797
1,STTRAM,ReadEDP,32,2.884000,10.922,5.070100e-11,1.021850e-10,0.568334,2.204000,4.536950,13422250.0,1251051.0,0.000681,0.000128,2.578503,2.579311
2,STTRAM,ReadDynamicEnergy,8,79.992000,88.485,2.876800e-11,7.480400e-11,0.093780,1.934000,4.870786,13422248.0,1251051.0,0.000386,0.000094,0.456782,0.457262
3,RRAM,ReadDynamicEnergy,8,4.672000,23.730,2.660100e-11,1.142810e-10,0.281180,0.352296,4.547864,13422237.0,1251051.0,0.000357,0.000143,1.278768,1.279268
4,STTRAM,Area,1,314.360000,322.387,5.736900e-11,8.109200e-11,0.027903,1.786000,6.472385,13422247.0,1251052.0,0.000770,0.000101,0.180599,0.181470
5,RRAM,ReadEDP,64,2.159000,20.670,3.099200e-11,6.588200e-11,0.377108,0.465897,4.533034,13422251.0,1251051.0,0.000416,0.000082,1.709443,1.709942
6,STTRAM,ReadLatency,512,2.659000,10.686,1.145980e-10,1.603930e-10,1.583000,2.503000,4.532669,13422252.0,1251051.0,0.001538,0.000201,7.175215,7.176954
7,PCRAM,ReadDynamicEnergy,64,4.542000,44.335,1.695300e-11,6.494000e-09,0.255420,0.467861,4.543840,13422237.0,1251051.0,0.000228,0.008124,1.160588,1.168939
8,RRAM,Area,4,11.586000,30.432,1.163350e-10,2.025640e-10,0.162125,0.202552,4.584833,13422248.0,1251051.0,0.001561,0.000253,0.743316,0.745131
9,PCRAM,ReadEDP,128,0.486710,40.304,2.103000e-11,6.502000e-09,0.473781,0.549767,4.525955,13422249.0,1251051.0,0.000282,0.008134,2.144311,2.152728
